In [39]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime, date

In [46]:
def url(ticker):
    return f'https://statusinvest.com.br/acoes/{ticker}'

In [47]:
def div_data(ticker):
    r = requests.get(url(ticker), headers={
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    })
    assert r.status_code == 200, f'error fetching: {r.text}'
    bs = BeautifulSoup(r.content)
    return bs.find_all('input', {'id': 'results'})[0]

In [48]:
def parse_divs(divs):
    df = pd.DataFrame(json.loads(divs_data['value']))
    df = df[['ed', 'et', 'v']].rename({'ed': 'com_date', 'et': 'div_type', 'v': 'value'}, axis=1)
    df['com_date'] = df['com_date'].map(lambda d: datetime.strptime(d, '%d/%m/%Y').date())
    return df.set_index('com_date').sort_index()

In [42]:
divs = div_data('itub4')
df = parse_divs(divs)
divs_2021 = df.loc[date(2021, 1, 1):date(2021, 12, 31)]

In [44]:
divs_2021.to_parquet('data/itub_divs_2021.parquet')

---

In [49]:
divs = div_data('vale3')
df = parse_divs(divs).loc[date(2016, 1, 1):date(2021, 12, 31)]
df

,div_type,value
com_date,,
2016-01-29,Dividendo,0.010000
2016-02-18,Dividendo,0.132000
2016-02-18,JCP,0.304267
2016-02-29,Dividendo,0.010000
2016-03-31,Dividendo,0.010000
...,...,...
2021-09-30,Dividendo,0.015000
2021-10-29,JCP,0.017650
2021-11-19,JCP,0.264551


In [50]:
df.to_parquet('data/vale_divs.parquet')